In [1]:
## import numpy as np # linear algebra
from matplotlib import pyplot as plt
#We need these in this file:
import sys
sys.path.append('cropping')
sys.path.append('scripts')
import load_read_name_extractor as lrne
import SVM_classifier_general as svm_general

#Taget fra længere nede i koden:
import torch
from torchvision import transforms
from PIL import Image
import random
import itertools
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import DeepFunctions as df
from random import shuffle
from torch.utils.data import Dataset, DataLoader, random_split
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import display
import os
import json
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
import torch.nn.functional as F

# Remember to change the cuda:0 value. Should match a free GPU on the cluster or if using local probably just cuda:0.

In [2]:
# Call the function to list GPU names
df.list_gpu_names()
#R
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Number of available GPUs: 4
GPU 0: NVIDIA TITAN X (Pascal)
GPU 1: NVIDIA TITAN X (Pascal)
GPU 2: NVIDIA TITAN X (Pascal)
GPU 3: NVIDIA TITAN X (Pascal)
cuda:0


In [3]:
trainDataloader = torch.load('data/train_loader.pth')
valDataloader = torch.load('data/val_loader.pth')
testDataloader = torch.load('data/test_loader.pth')

In [4]:
lenthdataset = torch.load('data/lengthsdataset.pth')

In [5]:
print(type(trainDataloader))
print(len(trainDataloader))
print(trainDataloader)

<class 'torch.utils.data.dataloader.DataLoader'>
117


In [6]:

#We have ignored class 9 since there were only 1 image with that class. Therfore we have 9 classes.
num_classes = 9

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3,stride=1):
        super(ResidualBlock, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection (identity mapping)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Add the shortcut connection
        out += self.shortcut(residual)
        out = self.relu(out)

        return out

In [8]:
kernel_size = 3
kernel_size1 = 3
padding = 0
# define network
conv1toconv2 = 16
conv2toconv3 = 32
conv3toconv4 = 64
conv4toconv5 = 128
conv5tores1 = 256
res1tores2 = 256
res2tores3 = 256
res3tores4 = 256
res4tores5 = 256
res5tores6 = 256
res6tolin1 = 256
lin = 256
momentum = 0.1

class NeuralNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(NeuralNetwork, self).__init__()
        
        self.conv1 = nn.Conv2d(3, conv1toconv2, kernel_size, padding=padding)
        self.bn1 = nn.BatchNorm2d(conv1toconv2)
        self.conv2 = nn.Conv2d(conv1toconv2, conv2toconv3, kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm2d(conv2toconv3)
        self.conv3 = nn.Conv2d(conv2toconv3, conv3toconv4, kernel_size, padding=padding)
        self.bn3 = nn.BatchNorm2d(conv3toconv4)
        self.conv4 = nn.Conv2d(conv3toconv4, conv4toconv5, kernel_size, padding=padding)
        self.bn4 = nn.BatchNorm2d(conv4toconv5)
        self.conv5 = nn.Conv2d(conv4toconv5, conv5tores1, kernel_size, padding=padding)
        self.bn5 = nn.BatchNorm2d(conv5tores1)
        
        self.residual1 = ResidualBlock(conv5tores1, res1tores2)
        self.residual2 = ResidualBlock(res1tores2, res2tores3)
        self.residual3 = ResidualBlock(res2tores3, res3tores4)
        self.residual4 = ResidualBlock(res4tores5, res4tores5)
        self.residual5 = ResidualBlock(res4tores5, res5tores6)
        self.residual6 = ResidualBlock(res5tores6, res6tolin1)
        
        self.pool = nn.AvgPool2d(2, 2)
        self.poolmax = nn.MaxPool2d(2,2)
        self.flatten = nn.Flatten()
        self.FC1 = nn.Linear(6 * res6tolin1, lin)
        self.bn7 = nn.BatchNorm1d(lin)
        self.FC2 = nn.Linear(lin,lin)
        self.bn8 = nn.BatchNorm1d(lin)
        self.FC3 = nn.Linear(lin, num_classes)
        self.bn9 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        #print(x.shape)
        x = self.poolmax(x)
        
        #print(x.shape)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        #print(x.shape)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)
        
                                  
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool(x)
        #print(x.shape)
        
        x = self.conv5(x)
        x = self.bn5(x)
        x = F.relu(x)
        x = self.pool(x)
        #print(x.shape)
        
        x = self.residual1(x)
        x = self.residual2(x)
        x = self.residual3(x)
        x = self.residual4(x)
        x = self.residual5(x)
        x = self.residual6(x)
        
        x = self.pool(x)
        
        x = self.flatten(x)
        x = self.FC1(x)
        x = self.bn7(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.FC2(x)
        x = self.bn8(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.FC3(x)
        return x

In [ ]:
#%%capture output
done=False
learning_rates = [0.0008]
patience_values = [5]
weight_decay_values =[0.000001]
label_smoothing_values =[0.3]
message = "Standard"
best_val_acc = 0.0
best_hyperparameters = {}

# Set the number of epochs
num_epochs = 140
totalAcc = 0

trainAccList = []
valAccList = []
trainLossList = []
valLossList = []

# Variables for early stopping
best_val_accuracy = 0.0
patience = 140  # Number of epochs to wait for improvement
counter = 0  # Counter for consecutive epochs without improvement
best_epoch = 0
best_model_state = None

# Training loop + tuning
for learning_rate in learning_rates:
    for weight_decay in weight_decay_values:
        for patience in patience_values:
            for label_smoothing in label_smoothing_values:
                net = NeuralNetwork()
                print("start")
                net.to(device)
                #print(net)
                #learning_rate = 0.001
                # Define the optimizer
                criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing).to(device)
                optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
                scheduler = ReduceLROnPlateau(optimizer, 'min', patience=patience, verbose=True)

                trainAccList = []
                valAccList = []
                trainLossList = []
                valLossList = []
                # Training loop
                for epoch in range(num_epochs):
                    running_loss = 0.0
                    totalAcc = 0.0
                    val_running_loss = 0.0
                    val_totalAcc = 0.0

                    for i, data in enumerate(trainDataloader, 0):
                        net.train()
                        # Get the inputs and labels
                        inputs, labels = data

                        inputs, labels = inputs.to(device), labels.to(device)

                        # Zero the parameter gradients
                        optimizer.zero_grad()

                        # Forward pass
                        outputs = net(inputs)
                        totalAcc += torch.eq(torch.argmax(outputs, dim=1), labels).sum()

                        loss = criterion(outputs, labels)

                        # Backward pass and optimization
                        loss.backward()
                        optimizer.step()

                        # Print statistics
                        running_loss += loss.item()
                        del(inputs)
                        del(labels)

                    for i, data in enumerate(valDataloader, 0):
                        net.eval()
                        # Get the inputs and labels
                        inputs, labels = data
                        data=None
                        inputs, labels = inputs.to(device), labels.to(device)

                        # Forward pass
                        outputs = net(inputs)
                        val_totalAcc += torch.eq(torch.argmax(outputs, dim=1), labels).sum()

                        loss = criterion(outputs, labels)

                        # Print statistics
                        val_running_loss += loss.item()
                        del(inputs)
                        del(labels)

                    scheduler.step(val_running_loss)

                    trainAccList.append(totalAcc/lenthdataset[0])
                    valAccList.append(val_totalAcc/lenthdataset[1])
                    trainLossList.append(running_loss/len(trainDataloader))
                    valLossList.append(val_running_loss/len(valDataloader))
                    print('[%d] train loss: %.3f train acc: %.3f val loss: %.3f val acc: %.3f '
                          % (epoch + 1,trainLossList[epoch],  trainAccList[epoch], valLossList[epoch], valAccList[epoch]))
                    running_loss = 0.0
                    totalAcc = 0.0
                    val_running_loss = 0.0
                    val_totalAcc = 0.0
                    current_learning_rate = optimizer.param_groups[0]['lr']

                    # Check for improvement in validation accuracy

                    if valAccList[epoch] > best_val_acc and valAccList[epoch] > 0.6:
                        best_val_acc = valAccList[epoch]
                        best_model_state = net.state_dict()
                        best_epoch = epoch

                    # Break if totalAcc is above 0.95
                    if trainAccList[epoch] > 1 or current_learning_rate < 1e-7:
                        print("Training accuracy reached above 0.95 or learning rate below 1e-6. Stopping training.")
                        break

                if best_model_state is not None:
                    net.load_state_dict(best_model_state)
                    torch.save(best_model_state, 'models/best_model_stateAcc%.3fepoch%.1f.pt' % (best_val_acc, best_epoch))
                    filename = 'models/best_model_stateAcc%.3fepoch%.1f.pt' % (best_val_acc, best_epoch)
                best_val_accuracy = 0.0
                best_epoch = 0
                best_model_state = None
                    

                print('Finished training')
                df.save_training_data_to_file(df.transfer_to_cpu([learning_rate,weight_decay,patience,momentum,label_smoothing,filename]),df.transfer_to_cpu(trainLossList)
                                                  ,df.transfer_to_cpu(trainAccList),df.transfer_to_cpu(valLossList)
                                                  ,df.transfer_to_cpu(valAccList))
done=True

start
[1] train loss: 2.064 train acc: 0.283 val loss: 1.981 val acc: 0.338 
[2] train loss: 1.994 train acc: 0.321 val loss: 1.947 val acc: 0.367 
[3] train loss: 1.951 train acc: 0.356 val loss: 1.909 val acc: 0.379 
[4] train loss: 1.933 train acc: 0.368 val loss: 1.902 val acc: 0.376 
[5] train loss: 1.912 train acc: 0.380 val loss: 1.875 val acc: 0.402 
[6] train loss: 1.906 train acc: 0.385 val loss: 1.891 val acc: 0.383 
[7] train loss: 1.888 train acc: 0.396 val loss: 1.859 val acc: 0.423 
[8] train loss: 1.874 train acc: 0.414 val loss: 1.884 val acc: 0.403 
[9] train loss: 1.874 train acc: 0.409 val loss: 1.846 val acc: 0.435 
[10] train loss: 1.857 train acc: 0.428 val loss: 1.823 val acc: 0.429 
[11] train loss: 1.843 train acc: 0.442 val loss: 1.801 val acc: 0.457 
[12] train loss: 1.844 train acc: 0.437 val loss: 1.841 val acc: 0.420 
[13] train loss: 1.831 train acc: 0.445 val loss: 1.802 val acc: 0.458 
[14] train loss: 1.825 train acc: 0.452 val loss: 1.805 val acc: 0.

In [ ]:
if done:
	output.show() # displays captured output
else:
	print("cell above still seem to be running, wait some more..")

In [ ]:
# Check the current GPU memory usage in bytes
current_memory = torch.cuda.memory_allocated(device=device)
print(f"Current GPU memory usage: {current_memory / (1024**3):.2f} GB")

# Check the GPU memory reserved by PyTorch (memory that can be freed, but is not returned to the OS)
reserved_memory = torch.cuda.memory_reserved(device=device)
print(f"GPU memory reserved by PyTorch: {reserved_memory / (1024**3):.2f} GB")